In [58]:
# Implementation of a very stripped down version of the C program aigsim

# Source Files
modelFile = 'aigTestSMV2.aag.txt'
stimFile  = 'stim1.txt'
# modelFile = 'aigTestLatch.aag.txt'
# stimFile  = 'stimELatch.txt'

STORE_PATH = '/Users/john/Jupyter/Machine Learning'

validInput = {"0","1"}


In [39]:
from dataclasses import dataclass
import math

class aiger_symbol:

    lit     = 0  # Number assigned to this gate as literal [0..2*maxvar+1]
    gID     = 0  # Gate ID number. Assumed use is to sequentially number gates of a same class
    type    = '' # What type of object is this    - aigsim new parameter
    oldVal  = 0  # The previous gate value        - aigsim new parameter
    curVal  = 0  # The current gate value
    gName   = ''
    myInput = '' # The gate that feeds this one
    myInputNeg = False # Is the feeder gate negated
	
    def __init__(self,lit,typeName,gID=0,gName=''):
        self.lit = lit
        self.type = typeName
        if gName == '':
            self.gName = typeName[0] + str(gID)
 
    def connect(self,gateList):
        self.myInput = gateList[int(self.lit/2)]
        if self.lit % 2 != 0:
            self.myInputNeg = True
          
    def prepStep(self):
        self.oldVal = self.curVal
        self.curVal = float('nan')
          
    def step(self):
        return(self.curVal)
        
    def printSelf(self):
        conStr = self.myInput.gName
        if self.myInputNeg == True:
           conStr += '*'
        else:
           conStr += ' '
           
        print('aiger_symbol - Type: {:6} lit: {:2}                    input: {:3}     name:{:10}'.format(self.type,self.lit,conStr,self.gName))                                


In [40]:
class aiger_const(aiger_symbol):

    def prepStep(self):
        self.oldVal = self.curVal

In [41]:
class aiger_input(aiger_symbol):

    pass

In [42]:
class aiger_output(aiger_symbol):

          
    def step(self):
        self.curVal = self.myInput.step()
        return(self.curVal)

In [43]:
class aiger_latch(aiger_symbol):

    next   = 0  # Number of gate connected to this gate's input as literal - latches only
    reset  = 0
    nextVal = 0

    def __init__(self,lit,input,reset,gID=0,gName=''):
        super().__init__(lit,'Latch',gID,gName)
        self.next = input
        self.reset = reset
        
    def connect(self,gateList):
        self.myInput = gateList[int(self.next/2)]
        if self.next % 2 != 0:
            self.myInputNeg = True

    def prepStep(self):
        self.oldVal = self.curVal
        self.curVal = float('nan')

    def step(self):
        if math.isnan(self.curVal):
            self.curVal  = self.nextVal

            self.nextVal = self.myInput.step()
            if self.myInputNeg == True:
                self.nextVal = int(bin(self.nextVal+1)[-1])
            
        return self.curVal
        
    def printSelf(self):
        conStr = self.myInput.gName
        if self.myInputNeg == True:
           conStr += '*'
        else:
           conStr += ' '
           
        print('aiger_symbol - Type: {:6} lit: {:2} next: {:2} reset: {:2} input: {:3}     name:{:10}'.format(self.type,self.lit,self.next,self.reset,conStr,self.gName))                                


In [44]:
class aiger_and(aiger_symbol):
    
    rhs0 = 0  # as literal [0..2*maxvar+1]
    rhs1 = 0  # as literal [0..2*maxvar+1] 
    in0  = 0
    in0Neg = False
    in1  = 0
    in1Neg = False

    def __init__(self,lit,rhs0,rhs1,gID=0,gName=''):
        super().__init__(lit,'And',gID,gName)
        self.rhs0 = rhs0
        self.rhs1 = rhs1

    def connect(self,gateList):
        self.in0 = gateList[int(self.rhs0/2)]
        if self.rhs0 % 2 != 0:
            self.in0Neg = True

        self.in1 = gateList[int(self.rhs1/2)]
        if self.rhs1 % 2 != 0:
            self.in1Neg = True

    def step(self):
        if math.isnan(self.curVal):
            rhs0 = self.in0.step()
            if self.in0Neg == True:
                rhs0 = int(bin(rhs0+1)[-1])
            rhs1 = self.in1.step()
            if self.in1Neg == True:
                rhs1 = int(bin(rhs1+1)[-1])
            self.curVal = rhs0 & rhs1
           
        return self.curVal

    def printSelf(self):        
        conStr = self.in0.gName
        if self.in0Neg == True:
           conStr += '*'
        else:
           conStr += ' '
           
        conStr += ' ' + self.in1.gName
        if self.in1Neg == True:
           conStr += '*'

        print('aiger_symbol - Type: {:6} lit: {:2} rhs0: {:2}  rhs1: {:2} input: {:7} name:{:10}'.format(self.type,self.lit,self.rhs0,self.rhs1,conStr,self.gName))


In [55]:
@dataclass
class Reader:
    
    inFile = ''
    
    def _init_(self):
        pass
    
    def openFile(self,file):
        
        self.inFile = open(file)
        
    def readHeader(self,model):
        
        args = (self.inFile.readline()).split()
        
        if args[0] != 'aag':
            return -1
        
        if len(args) < 6:
            sys.exit('Insufficient model parameters MILOA minimum requirement')
            
        model.maxvar      = int(args[1])
        model.num_inputs  = int(args[2])
        model.num_latches = int(args[3])
        model.num_outputs = int(args[4])
        model.num_ands    = int(args[5])
        
        if len(args) >= 7:
            model.num_bad = int(args[6])
            
        if len(args) >= 8:
            model.num_constraints = int(args[7])
       
        if len(args) >= 9:
            model.num_justice = int(args[8])

        if len(args) >= 10:
            model.num_fairness = int(args[9])

        return 0
    
    def validateInput(self,numArgs,errStr,verbose):
        
        args = (self.inFile.readline()).split()
        
        err = 0
        if len(args) < numArgs:
            print(errStr)
            err = -1
        
        if verbose == True:
            print(args)
            
        return args,err

#--------------------------------------------------------------------------------------

    def readModel(self,model):
        
        verbose = False
        gateList = [0] * (model.maxvar + 1)
        gateList[0] = aiger_const(0,'Constant',0)
        
        model.inputs = [0]*model.num_inputs
        for i in range(0,model.num_inputs):
            args,err = self.validateInput(1,'Invalid model definition - Input',verbose)
            if err == 0:
                model.inputs[i] = aiger_input(int(args[0]),'Input',i)
                gateList[int(int(args[0])/2)] = model.inputs[i]
                 
        model.latches = [0]*model.num_latches
        for i in range(0,model.num_latches):
            args,err = self.validateInput(2,'Invalid model definition - Latches',verbose)
            if err == 0:
                if len(args) == 2:
                    args.append('0')
                model.latches[i] = aiger_latch(int(args[0]),int(args[1]),int(args[2]),i)
                gateList[int(int(args[0])/2)] = model.latches[i]
        
        model.outputs = [0]*model.num_outputs
        for i in range(0,model.num_outputs):
            args,err = self.validateInput(1,'Invalid model definition - Output',verbose)
            if err == 0:
                model.outputs[i] = aiger_output(int(args[0]),'Output',i)
        
        # Read but ignore any bad states
        for i in range(0,model.num_bad):
            args,err = self.validateInput(1,'Invalid model definition - Bad State',verbose)
            
        # Read but ignore any constraints 
        for i in range(0,model.num_constraints):
            args,err = self.validateInput(1,'Invalid model definition - Constraints',verbose)

        # Read but ignore any justice properties 
        if model.num_justice > 0:
            for j in range(0,model.num_justice):
                tmp = (self.inFile.readline()).split()

            for j in range(0,model.num_justice):
                tmp1 = (self.inFile.readline()).split()
                tmp2 = (self.inFile.readline()).split()

        model.ands = [0]*model.num_ands
        for i in range(0,model.num_ands):
            args,err = self.validateInput(3,'Invalid model definition - Ands',verbose)
            if err == 0:
                model.ands[i] = aiger_and(int(args[0]),int(args[1]),int(args[2]),i)
                gateList[int(int(args[0])/2)] = model.ands[i]

        for i in range(0,model.num_inputs):
            model.inputs[i].connect(gateList)
  
        for i in range(0,model.num_latches):
            model.latches[i].connect(gateList)

        for i in range(0,model.num_outputs):
            model.outputs[i].connect(gateList)

        for i in range(0,model.num_ands):
            model.ands[i].connect(gateList)

#--------------------------------------------------------------------------------------
              
    def getStim(self):
        
        args = (self.inFile.readline()).split()
        
        return args

#--------------------------------------------------------------------------------------


In [49]:
@dataclass
class Model:

    stepNum         = 0
    maxvar          = 0
    num_inputs      = 0
    num_latches     = 0
    num_outputs     = 0
    num_ands        = 0
    num_bad         = 0
    num_constraints = 0
    num_justice     = 0
    num_fairness    = 0

    inputs  = [] # [0..num_inputs]
    latches = [] # [0..num_latches]
    outputs = [] # [0..num_outputs]

    ands    = [] # [0..num_ands]
    
    current = [] # [0..maxvar+1] - holds current output of each gate

    def _init_(self):
        pass
    
    def initModel(self):
        self.stepNum = 0
        self.current = [0] * (self.maxvar + 1) # for index simplicity, index 0 is unused
        
        # Initialize the latches    
        # - Note this code does not support reset reset values outside {0,1}.
        
        for i in range(0,self.num_latches):
            self.current[int((self.latches[i].lit)/2)] = self.latches[i].reset
        
    # Does not support ground or don't care values
    
    def validateInput(self,args):
        
        err = 0
        if len(args) == self.num_inputs:
            current = [0] * self.num_inputs
            for i in range (self.num_inputs):
                if validInput.issuperset(args.rstrip()):   
                    current[i] = int(args[i])
                else:
                    print('invalid characters in input string')
                    err = -1
                    
        else:
            print('invalid input string length')
            err = -1

        return current,err
        
    def getCurVal(self,lit):
        val = self.current[int(lit/2)]
        if lit%2 != 0:
            val = int(bin(val+1)[-1])
        
        return val
    
    def step(self,args,verbose):
        
        for i in range(0,self.num_inputs):
            self.inputs[i].prepStep()
            
        for i in range(0,self.num_latches):
            self.latches[i].prepStep()

        for i in range(0,self.num_ands):            
            self.ands[i].prepStep()
        
        stim,err = self.validateInput(args)  

        # Process the input stimuli
        for i in range(0,self.num_inputs):
            self.inputs[i].curVal = stim[i]

        # Process the latches
        for i in range(0,self.num_latches):
            self.latches[i].step()

        # Process the and gates
        for i in range(0,self.num_ands):
            self.ands[i].step()
            
         # Process the output gates
        for i in range(0,self.num_outputs):
            self.outputs[i].step()

        if verbose == True:
            self.printState(self.stepNum)
        self.stepNum += 1
    
    def printResults(self):
        
        for i in range(0,self.num_latches):
            print('{:1}'.format(self.latches[i].curVal),end='')
        print(' ',end='')

        for i in range(0,self.num_inputs):
            print('{:1}'.format(self.inputs[i].curVal),end='')
        print(' ',end='')
            
        for i in range(0,self.num_outputs):
            print('{:1}'.format(self.outputs[i].curVal),end='')
        print(' ',end='')
        
        for i in range(0,self.num_latches):
            print('{:1}'.format(self.latches[i].nextVal),end='')

        print('')
        
    def printSelf(self):
        print('Model')
        print('-----')
        print('maxvar      = ',self.maxvar)
        print('num_inputs  = ',self.num_inputs)
        print('num_latches = ',self.num_latches)
        print('num_outputs = ',self.num_outputs)
        print('num_ands    = ',self.num_ands)
        
        for i in range(0,self.num_inputs):
            self.inputs[i].printSelf()
            
        for i in range(0,self.num_latches):
            self.latches[i].printSelf()
            
        for i in range(0,self.num_outputs):
            self.outputs[i].printSelf()
            
        for i in range(0,self.num_ands):
            self.ands[i].printSelf()
        
    def printState(self,stepNum=0):
    
        print("{:4d} ".format(stepNum),end='')
        for i in range(0,self.num_latches):
            print("{:1d}".format(self.latches[i].curVal),end='')
        print(' ',end='')
            
        for i in range(0,self.num_inputs):
            print("{:1d}".format(self.inputs[i].curVal),end='')
        print(' ',end='')
            
        for i in range(0,self.num_latches):
            print("{:1d}".format(self.latches[i].nextVal),end='')
        print(' ',end='')
            
        for i in range(0,self.num_outputs):
            print("{:1d}".format(self.outputs[i].curVal),end='')
        print(' ',end='')
            
        for i in range(0,self.num_ands):
            print("{:1d}".format(self.ands[i].curVal),end='')
            
        print('')


In [59]:
verbose0 = True
verbose1 = False
verbose2 = True

model = Model()

reader = Reader()
reader.openFile(modelFile)
reader.readHeader(model)
reader.readModel(model)

if verbose0 == True:
    model.printSelf()
    
model.initModel()

reader = Reader()
reader.openFile(stimFile)

done = False

while done != True:
    stim = reader.getStim()
    if len(stim) > 0:
        if stim[0] == '.':
            done = True
        else:
            model.step(stim[0],verbose2)
            if verbose == verbose1:
                model.printResults()
            
    else:
        print('Stim file not properly terminated. Last line should only contain a period')
        done = True



Model
-----
maxvar      =  7
num_inputs  =  2
num_latches =  2
num_outputs =  1
num_ands    =  3
aiger_symbol - Type: Input  lit:  2                    input: I0      name:I0        
aiger_symbol - Type: Input  lit:  4                    input: I1      name:I1        
aiger_symbol - Type: Latch  lit: 12 next:  6 reset:  0 input: A0      name:L0        
aiger_symbol - Type: Latch  lit: 10 next:  8 reset:  0 input: A1      name:L1        
aiger_symbol - Type: Output lit: 14                    input: A2      name:O0        
aiger_symbol - Type: And    lit:  6 rhs0:  2  rhs1:  4 input: I0  I1  name:A0        
aiger_symbol - Type: And    lit:  8 rhs0:  4  rhs1:  7 input: I1  A0* name:A1        
aiger_symbol - Type: And    lit: 14 rhs0: 11  rhs1: 12 input: L1* L0  name:A2        
   0 00 11 10 0 100
   1 10 11 10 1 101
   2 10 10 00 1 001
   3 00 10 00 0 000
   4 00 11 10 0 100
   5 10 00 00 1 001
   6 00 10 00 0 000
   7 00 11 10 0 100
   8 10 01 01 1 011
   9 01 01 01 0 010
  10 01 11 10 0

In [ ]:
a = 1
b = bin(a+1)[-1]
print(a+1)
print(int(b))
print(type(b))
print(b[-1])

In [ ]:
import math

x = float('nan')
math.isnan(x)
